In [8]:
import tensorflow as tf
from tensorflow import keras

print(pd.__version__)
print(np.__version__)
print(tf.__version__)
print(keras.__version__)

1.3.3
1.21.2
2.4.1
2.4.0


In [4]:
def generate_noun_summary(prompt_text):
    
    try:
        messages = [{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": prompt_text}]

        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages
        )
        return str(response['choices'][0]['message']['content']).strip()
    
    except:
        time.sleep(10)

In [5]:
def create_unique_word_dict(text:list) -> dict:
    """
    A method that creates a dictionary where the keys are unique words
    and key values are indices
    """
    # Getting all the unique words from our text and sorting them alphabetically
    words = list(set(text))
    words.sort()

    # Creating the dictionary for the unique words
    unique_word_dict = {}
    for i, word in enumerate(words):
        unique_word_dict.update({
            word: i
        })

    return unique_word_dict

In [7]:
import itertools
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm

# 딥러닝 관련
from tensorflow.python.keras.models import Input, Model
from tensorflow.python.keras.layers import Dense
from scipy.sparse import csr_matrix

# gpt api 관련
import openai
import concurrent.futures
API_KEY = ''
openai.api_key = API_KEY

In [13]:
# ▶ Warnings 제거
import warnings
warnings.filterwarnings('ignore')

In [14]:
df = pd.read_excel('./data/data.xlsx')
df.head(5)

,title
0,금융관련 모바일 앱 디자인
1,금융거래웹사이트 디자인개편
2,금융사 네이티브 모바일앱 디자인
3,금융사 앱 개발 프로젝트
4,금융정보 비교앱 개발


In [15]:
# gpt-4 API로 데이터 전처리

texts = []

for i in tqdm(range(0, len(df), 10)):
    
    ii = i + 10
    
    extracted_description = df[i:ii]['title']
    
    input_texts = []
    
    count = 0
    for index, result in enumerate(extracted_description):
        
        input_texts.append("""이전의 대화내용은 잊어버리고 텍스트분석을 위해 다음 ; 뒤의 텍스트에서 불용어를 제외하고 산업 또는 IT와 관련된 유의미한 명사만 남겨줘. 접두사와 접미사는 제외하고 명사만 남겨줘. 예를 들어, '금융권'과 '금융사'는 '금융'으로 치환해줘. 단, '금융감독원', '금융상품'과 같이 독립적인 의미를 갖는 단어들은 수정하면 안돼. 주어진 텍스트에서만 명사를 뽑아줘.; {}.""".format(result))
        
    with concurrent.futures.ThreadPoolExecutor() as executor:

        results = executor.map(generate_noun_summary, input_texts)

    for index, result in enumerate(results):
        original_list = result[1:-1].split(',')
        texts.append([word.replace(' ', '').replace("'", "").replace('"', '') for word in original_list])

100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [05:32<00:00,  7.40s/it]


In [41]:
temp = pd.DataFrame(columns=['text'])
for index, text in enumerate(texts):
    temp.at[index, 'text'] = text

In [43]:
temp.to_csv('processed_by_gpt4.csv', index=False)

In [16]:
for index, result in enumerate(results):
    original_list = result[1:-1].split(',')
    texts.append([word.replace(' ', '') for word in original_list])

In [17]:
# 데이터 전처리 수작업
# 특정단어 같은 글꼴로 치환

change_word = {'관리앱':'앱', '백엔':'backend', '비금육':'비금융', '려동물':'반려동물', '험사':'보험사', '사:금융상품':'금융상품', 'K금융':'금융', '국계':'외국계', 'Frontend':'frontend', '사:금융투자':'금융투자', '엑트':'react', '뉴얼':'리뉴얼', '리엑트':'react', '상자산':'자산', 'eactnative':'react', 'JAVA':'java', '민은행':'은행', '험':'보험', '동인증서':'공동인증서', '외':'해외', 'B금융':'금융', '세대':'차세대', '행':'은행', '한금융투자':'금융투자', '식데이터':'데이터', '록체인':'블록체인', '사:앱':'앱', '부산]':'부산', '이터':'데이터', '석된결과':'', '사:상용':'상용', '바일뱅킹':'모바일뱅킹', '융기관':'금융', 'AI개발':'ai', 'AVA':'java', 'eact':'react', '금융투자자':'금융투자', '융투자':'금융투자', 'PM입니다':'pm', 'ava':'java', '하나금융':'금융', '팅':'채팅', '산관리':'자산관리', '공업무':'공공업무', '영':'운영', '미디':'미디어', '제':'제작', '구':'구축', '개':'개편', '프로젝':'프로젝트', '융':'금융', '금육':'금융', '융':'금융', '디자':'디자인', '2P':'2p', 'PC':'pc', '웹사이트':'웹', '융권':'금융', '프로젝':'프로젝트', 'API':'api', '융사':'금융', '퍼블리':'퍼블리싱', 'DB':'db', 'IT':'it', 'Java':'java', '금융사':'금융', '데스크탑':'pc', '리뉴':'리뉴얼', '반응형':'웹', '백엔드':'backend', '안드로이드':'앱', '어플리케이션':'앱', '웹서비스':'웹', '웹페이지':'웹', '융결제원':'금융결제원', '융상품':'금융상품', '프런트엔드':'frontend', '플랫':'플랫폼', '2p금':'2p', '2페이지':'2p', 'AI개':'ai', 'APP':'앱', 'App':'앱', 'Front-end':'frontend', 'HP':'hp', 'IOS':'앱', 'OTP':'otp', 'SQL':'sql', 'UI':'ui', 'e-금융고도':'e-금융', 'evExpress':'DevExpress', 'ySQL':'mysql', '금융관련':'금융', '금융권':'금융', '모바일앱':'앱', '모바일웹':'웹', '비교앱':'앱', '분석웹':'웹', '인공지능':'ai'}
change_word_keys = list(change_word.keys())

# 고칠 수 있는 단어는 고치는 코드
temp_texts = [[change_word[word] if word in change_word_keys else word for word in text] for text in texts]

# 한 글자만 있거나 8글자이상인 이상하게 전처리된 리스트 제거 (gpt의 잘못된 전처리로 인해 생긴 리스트는 버리기)
# 단 한 글자 또는 8글자가 의미가 있을 땐 계속 추가 
words_to_check = ['웹', '앱', 'Objective-C', 'frontend', 'backend']

modified_texts = []
for temp_text in temp_texts:
    
    temp = []
    check = 0
    
    for word in temp_text:
        
        if len(word) > 1 and len(word) <= 7:
            temp.append(word)
        else:
            if word in words_to_check:
                temp.append(word)
                continue
                
            else:
                check = 1
                break
                
    if check == 0:
        modified_texts.append(temp)

In [18]:
all_texts = []

for text in modified_texts:
    for word in text:
        all_texts.append(word)

In [19]:
# 고유 단어 갯수가 몇개인지.. --> 고유 단어 갯수만큼 인풋 노드들이 생김 
df_count = pd.DataFrame(all_texts, columns=['title'])
df_count = df_count.groupby(by='title', as_index=False).agg(count_title=('title','count'))
len(df_count)

483

In [20]:
# 근처단어를 탐색할 window 정해주고 근처단어 구하기
window = 4

word_lists = []
all_text = []

for text in modified_texts:

    for word in text:
        all_text += word.split()

    for i, word in enumerate(text):
        for w in range(window):
            
            if i + 1 + w < len(text):
                word_lists.append([word] + [text[(i + 1 + w)]])

            if i - w - 1 >= 0:
                word_lists.append([word] + [text[(i - w - 1)]])

In [21]:
unique_word_dict = create_unique_word_dict(all_text)
n_words = len(unique_word_dict)
words = list(unique_word_dict.keys())

In [22]:
# 데이터 만드는 부분. 만약 총 단어 갯수가 5개이고 고양이가 2번째에서 나타나고 그에 대응하는 통조림이 3번쨰에서 나타났다면 
# X = [0, 1, 0, 0, 0]
# Y = [0, 0, 1, 0, 0]

X = []
Y = []

for i, word_list in tqdm(enumerate(word_lists)):
    
    main_word_index = unique_word_dict.get(word_list[0])
    context_word_index = unique_word_dict.get(word_list[1])

    X_row = np.zeros(n_words)
    Y_row = np.zeros(n_words)

    X_row[main_word_index] = 1
    Y_row[context_word_index] = 1

    X.append(X_row)
    Y.append(Y_row)

9196it [00:00, 114705.69it/s]


In [23]:
X = np.array(X)
Y = np.array(Y)

In [24]:
# 희소 벡터 보완
sc_X = csr_matrix(X)
sc_Y = csr_matrix(Y)

In [25]:
# # 모델 정의
embed_size = 2

inp = Input(shape=(X.shape[1],), dtype='float64') # 21개의 노드
x = Dense(units=embed_size, activation='linear', dtype='float64')(inp) # 2개의 노드
x = Dense(units=Y.shape[1], activation='softmax', dtype='float64')(x) # 1개의 노드
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [26]:
# 모델 학습
model.fit(
    x=sc_X,
    y=Y,
    batch_size=256,
    epochs=1000
    )

weights = model.get_weights()[0]

Epoch 1/1000
36/36 [==============================] - 1s 4ms/step - loss: 6.1713
Epoch 2/1000
36/36 [==============================] - 0s 4ms/step - loss: 6.1337
Epoch 3/1000
36/36 [==============================] - 0s 4ms/step - loss: 6.0854
Epoch 4/1000
36/36 [==============================] - 0s 5ms/step - loss: 6.0153
Epoch 5/1000
36/36 [==============================] - 0s 4ms/step - loss: 5.9295
Epoch 6/1000
36/36 [==============================] - 0s 4ms/step - loss: 5.8198
Epoch 7/1000
36/36 [==============================] - 0s 4ms/step - loss: 5.7096
Epoch 8/1000
36/36 [==============================] - 0s 5ms/step - loss: 5.5836
Epoch 9/1000
36/36 [==============================] - 0s 4ms/step - loss: 5.4762
Epoch 10/1000
36/36 [==============================] - 0s 4ms/step - loss: 5.3508
Epoch 11/1000
36/36 [==============================] - 0s 4ms/step - loss: 5.2344
Epoch 12/1000
36/36 [==============================] - 0s 4ms/step - loss: 5.1538
Epoch 13/1000
36/36 [====

36/36 [==============================] - 0s 5ms/step - loss: 4.7610
Epoch 102/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.7579
Epoch 103/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.7435
Epoch 104/1000
36/36 [==============================] - 0s 6ms/step - loss: 4.7680
Epoch 105/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.7657
Epoch 106/1000
36/36 [==============================] - 0s 7ms/step - loss: 4.7287
Epoch 107/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.7328
Epoch 108/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.7547
Epoch 109/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.7576
Epoch 110/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.7453
Epoch 111/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.7647
Epoch 112/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.7343
Epoch 113/1000
36/3

36/36 [==============================] - 0s 4ms/step - loss: 4.5991
Epoch 298/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.5885
Epoch 299/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.6057
Epoch 300/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.6016
Epoch 301/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.6107
Epoch 302/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.6195
Epoch 303/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.6077
Epoch 304/1000
36/36 [==============================] - 0s 6ms/step - loss: 4.5653
Epoch 305/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.5845
Epoch 306/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.6066
Epoch 307/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.6082
Epoch 308/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.6060
Epoch 309/1000
36/3

36/36 [==============================] - 0s 4ms/step - loss: 4.5439
Epoch 494/1000
36/36 [==============================] - 0s 6ms/step - loss: 4.5314
Epoch 495/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.5470
Epoch 496/1000
36/36 [==============================] - 0s 6ms/step - loss: 4.5376
Epoch 497/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.5413
Epoch 498/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.5285
Epoch 499/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.5088
Epoch 500/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.5324
Epoch 501/1000
36/36 [==============================] - 0s 3ms/step - loss: 4.5205
Epoch 502/1000
36/36 [==============================] - 0s 3ms/step - loss: 4.5295
Epoch 503/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.5196
Epoch 504/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.5373
Epoch 505/1000
36/3

36/36 [==============================] - 0s 7ms/step - loss: 4.4821
Epoch 690/1000
36/36 [==============================] - 0s 6ms/step - loss: 4.5104
Epoch 691/1000
36/36 [==============================] - 0s 6ms/step - loss: 4.5155
Epoch 692/1000
36/36 [==============================] - 0s 7ms/step - loss: 4.4867
Epoch 693/1000
36/36 [==============================] - 0s 6ms/step - loss: 4.4669
Epoch 694/1000
36/36 [==============================] - 0s 6ms/step - loss: 4.5149
Epoch 695/1000
36/36 [==============================] - 0s 7ms/step - loss: 4.4680
Epoch 696/1000
36/36 [==============================] - 0s 7ms/step - loss: 4.5165
Epoch 697/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.4747
Epoch 698/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.4762
Epoch 699/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.4792
Epoch 700/1000
36/36 [==============================] - 0s 6ms/step - loss: 4.4854
Epoch 701/1000
36/3

36/36 [==============================] - 0s 4ms/step - loss: 4.4914
Epoch 788/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.4849
Epoch 789/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.4670
Epoch 790/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.4949
Epoch 791/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.4835
Epoch 792/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.4782
Epoch 793/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.4916
Epoch 794/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.5063
Epoch 795/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.4679
Epoch 796/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.4722
Epoch 797/1000
36/36 [==============================] - 0s 4ms/step - loss: 4.4836
Epoch 798/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.4960
Epoch 799/1000
36/3

36/36 [==============================] - 0s 6ms/step - loss: 4.4404
Epoch 984/1000
36/36 [==============================] - 0s 5ms/step - loss: 4.4666
Epoch 985/1000
36/36 [==============================] - 0s 7ms/step - loss: 4.4504
Epoch 986/1000
36/36 [==============================] - 0s 6ms/step - loss: 4.4668
Epoch 987/1000
36/36 [==============================] - 0s 7ms/step - loss: 4.4618
Epoch 988/1000
36/36 [==============================] - 0s 6ms/step - loss: 4.4801
Epoch 989/1000
36/36 [==============================] - 0s 7ms/step - loss: 4.4780
Epoch 990/1000
36/36 [==============================] - 0s 7ms/step - loss: 4.4536
Epoch 991/1000
36/36 [==============================] - 0s 7ms/step - loss: 4.4433
Epoch 992/1000
36/36 [==============================] - 0s 8ms/step - loss: 4.4391
Epoch 993/1000
36/36 [==============================] - 0s 9ms/step - loss: 4.4323
Epoch 994/1000
36/36 [==============================] - 0s 7ms/step - loss: 4.4581
Epoch 995/1000
36/3

In [27]:
embedding_dict = {}
for word in words: 
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
        })

In [28]:
# 시각화용 df 만들기
df_visual = pd.DataFrame(columns=['word','x','y'])

for key, values in embedding_dict.items():
    temp_df = pd.DataFrame(data=[[key, values[0], values[1]]], columns=['word','x','y'])
    df_visual = df_visual.append(temp_df, ignore_index=True)


In [29]:
index = df_visual[df_visual['word'] == '금융'].index
df_visual['hue'] = 0
df_visual.at[int(index[0]), 'hue'] = 1

In [30]:
import seaborn as sns
import matplotlib.pyplot as plt

# 한글 깨짐 방지
import os

if os.name == 'posix': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif os.name == 'nt': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

# 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

In [9]:
# # Ploting the embeddings
# # 금융이라는 단어를 지정해주고 빨강색으로 만들기 나머진 파랑

# word_to_mark = "금융"

# plt.figure(figsize=(30, 30))
# for word in list(unique_word_dict.keys()):
    
#     coord = embedding_dict.get(word)
#     if word == word_to_mark:
#         plt.scatter(coord[0], coord[1], c='red', s=100)  # Mark with red dot and larger size
#     else:
#         plt.scatter(coord[0], coord[1], c='blue', s=50)  # Others with blue dot and smaller size
#     plt.annotate(word, (coord[0], coord[1]))

In [32]:
# 유클리디안 거리 측정
def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))

In [33]:
euclidean_dist = {}

for word in unique_word_dict.keys():
    if word != '금융':
        x = embedding_dict['금융']
        y = embedding_dict[word]
        euclidean_dist[word] = euclidean_distance(x, y)
        
    else:
        continue

In [34]:
euclidean_df = pd.DataFrame.from_dict(data=euclidean_dist, orient='index', columns=['val']).reset_index().sort_values(by=['val'])

In [10]:
# euclidean_df.head(60).reset_index(drop=True)